In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import os 
import tqdm
import glob
import tensorflow 

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage import io
from skimage.color import rgb2gray

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

In [20]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,)

test_datagen  = ImageDataGenerator(rescale = 1./255)

In [21]:
train_dataset  = train_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Alzheimer_s Dataset/train',
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   batch_size = 32)

Found 5131 images belonging to 4 classes.


In [22]:
test_dataset  = test_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Alzheimer_s Dataset/test',
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   batch_size = 32)

Found 1279 images belonging to 4 classes.


In [24]:
model = Sequential()
model.add(Conv2D(filters = 64, padding = "same",activation = "relu",kernel_size=3, strides = 2,input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2),strides = 2))

model.add(Conv2D(filters = 64, padding = "same",activation = "relu",kernel_size=3))
model.add(MaxPool2D(pool_size=(2,2),strides = 2))

model.add(Conv2D(filters = 64, padding="same", activation="relu",kernel_size=3))
model.add(MaxPool2D(pool_size=(2, 2)))
...
model.add(Conv2D(filters = 128, padding="same", activation="relu",kernel_size=3))
model.add(MaxPool2D(pool_size=(2, 2)))
...
model.add(Conv2D(filters = 128, padding="same", activation="relu",kernel_size=3))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256,activation="relu"))

#Output layer
model.add(Dense(1,kernel_regularizer=l2(0.01),activation = "softmax"))

In [25]:
model.compile(optimizer = 'adam', loss = 'squared_hinge', metrics = ['accuracy'])

In [ ]:
model_history = model.fit(x = train_dataset, validation_data = test_dataset, epochs=15)

Epoch 1/15
161/161 [==============================] - 333s 2s/step - loss: 3.0047 - accuracy: 0.2500 - val_loss: 3.0003 - val_accuracy: 0.2500
Epoch 2/15
161/161 [==============================] - 330s 2s/step - loss: 3.0001 - accuracy: 0.2500 - val_loss: 3.0000 - val_accuracy: 0.2500
Epoch 3/15
161/161 [==============================] - 333s 2s/step - loss: 3.0000 - accuracy: 0.2500 - val_loss: 3.0000 - val_accuracy: 0.2500
Epoch 4/15
161/161 [==============================] - 335s 2s/step - loss: 3.0000 - accuracy: 0.2500 - val_loss: 3.0000 - val_accuracy: 0.2500
Epoch 5/15
161/161 [==============================] - 331s 2s/step - loss: 3.0000 - accuracy: 0.2500 - val_loss: 3.0000 - val_accuracy: 0.2500
Epoch 6/15
161/161 [==============================] - 331s 2s/step - loss: 3.0000 - accuracy: 0.2500 - val_loss: 3.0000 - val_accuracy: 0.2500
Epoch 7/15
161/161 [==============================] - 330s 2s/step - loss: 3.0000 - accuracy: 0.2500 - val_loss: 3.0000 - val_accuracy: 0.2500

In [ ]:
scores = model.evaluate_generator(test_dataset)
print("Accuracy = ", scores[1]*100)
print("Precision = ", scores[2])
print("Recall = ", scores[3])
print("AUC = ", scores[4])